<center><br> <img src="https://www.iscte-iul.pt/assets/images/logo_iscte_detailed.svg" style="width: 300px;"></center><br>

<img src="https://logos-world.net/wp-content/uploads/2021/02/Expedia-Emblem.png" style="width: 300px;margin-top: 30px;" align="left">
   <div><center><h1>Projeto: Expedia Hotel Recommendations</h1></center></div>
   <div><center><h4>Processamento de Big Data  -   2º Semestre | 2022/23</h4></center></div>
   <div><center><b>Docentes:</b> João Oliveira <b>|</b> Adriano Lopes</center></div>
 <br>
    <div><center><h4>Grupo 17</h4></center> </div>
    <div><center> André Silvestre Nº104532 <b>|</b> Rita Matos Nº104936 <b>CDB1</b></center> </div>
 <br>    
 <br>
 <br>
 <br>

<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: linear-gradient(to right, #075924, #067572, #0371B1, #001D45); 
            padding: 15px; color: white; border-radius: 30000px; text-align: center;">
    <center><h1 style="color: white;font-size: 30px; font-family: 'Avenir Next LT Pro', sans-serif;"><b>4 | Modelos de ML</b></h1></center>
</div>

### PySpark Session

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Expedia Project') \
    .config("spark.driver.memory", "10g") \
    .config("spark.executor.memory", "10g") \
    .config("spark.driver.host", "127.0.0.1") \
    .getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1680809728276_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# 📚 Import das Bibliotecas

In [2]:
# Imports Gerais
import os
import sys

# Imports Python
import numpy as np
import pandas as pd  
import warnings
warnings.filterwarnings("ignore")

# Imports PySpark
import pyspark.sql.functions as F
from pyspark.sql.types import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# 🧮 Importar Bases de Dados (parquet)

In [3]:
# Leitura da base de dados "df-expedia-3" guardada anteriormente
# df_expedia = spark.read.parquet('df-expedia-3')
df_expedia = spark.read.parquet('s3://pdb-grupo17-bucket-as/df-expedia-3')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Leitura da base de dados "small-df-expedia" guardada anteriormente
# small_df_expedia = spark.read.parquet('small-df-expedia')
small_df_expedia = spark.read.parquet('s3://pdb-grupo17-bucket-as/small-df-expedia')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Confirmação do nº de observações e tipos de dados
print("Nº de Observações:", df_expedia.count())
print("Nº de Observações:", small_df_expedia.count())

df_expedia.printSchema()
small_df_expedia.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Nº de Observações: 37455577
Nº de Observações: 11236139
root
 |-- srch_destination_id: integer (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable = true)
 |-- user_location_country: integer (nullable = true)
 |-- is_package: integer (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- is_booking: integer (nullable = true)
 |-- hotel_continent: integer (nullable = true)
 |-- hotel_country: integer (nullable = true)
 |-- hotel_market: integer (nullable = true)
 |-- pcaFeature_1: double (nullable = true)
 |-- pcaFeature_2: double (nullable = true)
 |-- pcaFeature_3: double (nullable = true)
 |-- tempo_estadia: integer (nullable = true)
 |-- hotel_cluster: integer (nullable = true)

root
 |-- srch_destination_id: integer (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable =

---

# ⚖️ Divisão em Conjunto Treino/Teste

In [8]:
# Divisão em Conjunto Treino/Teste
df_train, df_test = small_df_expedia.randomSplit([0.9, 0.1], 42) # Computador Local
df_train_aws, df_test_aws = df_expedia.randomSplit([0.9, 0.1], 42)     # AWS

n_df_train = df_train.count()
n_df_test = df_test.count()

n_df_train_aws = df_train_aws.count()
n_df_test_aws = df_test_aws.count()

# Nº de observações em cada parte
print(f"Existem {n_df_train} observações no Conjunto de Treino e {n_df_test} no Conjunto de Teste.")
print(f"Existem {n_df_train_aws} observações no Conjunto de Treino e {n_df_test_aws} no Conjunto de Teste. (AWS)")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Existem 10110707 observações no Conjunto de Treino e 1125432 no Conjunto de Teste.
Existem 33708161 observações no Conjunto de Treino e 3747416 no Conjunto de Teste. (AWS)

---

# 💡 Modelos do Spark ML

- **Random Forest** - Algoritmo de ML que usa várias árvores de decisão para classificar dados.
- **Decision Forest** (Árvore de Decisão) - Algoritmo de ML que usa uma estrutura de árvore para fazer previsões a partir de dados.

In [9]:
# Lista das features a usar
features_list = ['srch_destination_id', 'site_name', 'posa_continent','user_location_country', 
                 'is_package', 'srch_adults_cnt','srch_children_cnt', 'srch_destination_type_id', 
                 'is_booking','hotel_continent', 'hotel_country', 'hotel_market',
                 'pcaFeature_1', 'pcaFeature_2', 'pcaFeature_3','tempo_estadia']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Random Forest [[1]](https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.ml.classification.RandomForestClassifier.html) [[3]](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html)

<div class="alert alert-block alert-warning">
Amostra do Dataset | <b>Computador</b></div>

- Com o `small_df_expedia` (no **Computador Local**)

In [10]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

# Criar um VectorAssembler para concatenar as colunas para o modelo
assembler = VectorAssembler(inputCols=features_list, outputCol = 'features')

# Criar um classificador RandomForest
rndfr_sp = RandomForestClassifier(featuresCol='features', labelCol='hotel_cluster')

# Pipeline com o VectorAssembler e o classificador OneVsRest
pipeline = Pipeline(stages=[assembler, rndfr_sp])

# Treinar o modelo e prever as classes do conjunto de teste
model = pipeline.fit(df_train)
df_pred1_cl = model.transform(df_test)                              # cl - Computador Locar

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Visualização dos Resultados
df_pred1_cl.select('features', 'rawPrediction', 'prediction', 'hotel_cluster').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----------+-------------+
|            features|       rawPrediction|prediction|hotel_cluster|
+--------------------+--------------------+----------+-------------+
|[4.0,2.0,3.0,214....|[0.03064380352854...|      64.0|           30|
|[4.0,2.0,3.0,214....|[0.03064380352854...|      64.0|           67|
|[8.0,2.0,3.0,66.0...|[0.21657679305846...|      91.0|           77|
|[8.0,2.0,3.0,66.0...|[0.21657679305846...|      91.0|           16|
|[8.0,2.0,3.0,66.0...|[0.21657679305846...|      91.0|           94|
|[8.0,2.0,3.0,66.0...|[0.21657679305846...|      91.0|            7|
|[9.0,17.0,1.0,133...|[0.03532103558989...|      82.0|           20|
|[9.0,17.0,1.0,133...|[0.03532103558989...|      82.0|           40|
|[11.0,2.0,3.0,66....|[0.18765358208901...|      91.0|           91|
|[16.0,2.0,3.0,66....|[0.21657679305846...|      91.0|            7|
|[19.0,2.0,3.0,66....|[0.03426725529612...|      82.0|           75|
|[19.0,2.0,3.0,66....|[0.034267255

In [12]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Define o Evaluator
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='hotel_cluster', metricName='accuracy')

# Calcula a accuracy, precisão, recall e f1-score
accuracy = evaluator.evaluate(df_pred1_cl)
precision = evaluator.evaluate(df_pred1_cl, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(df_pred1_cl, {evaluator.metricName: "weightedRecall"})
f1_score = evaluator.evaluate(df_pred1_cl, {evaluator.metricName: "f1"})

print("Accuracy = ", accuracy)
print("Precision = ", precision)
print("Recall = ", recall)
print("F1-Score = ", f1_score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy =  0.09995663255331175
Precision =  0.0635198632841171
Recall =  0.10021396228292778
F1-Score =  0.04213159943428581

---

<div class="alert alert-block alert-danger">
Dataset Completo | <b>AWS</b></div>

- Com o `df_expedia` (no **AWS**)

In [13]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

# Criar um VectorAssembler para concatenar as colunas para o modelo
assembler = VectorAssembler(inputCols=features_list, outputCol = 'features')

# Criar um classificador RandomForest
rndfr_sp = RandomForestClassifier(featuresCol='features', labelCol='hotel_cluster')

# Pipeline com o VectorAssembler e o classificador OneVsRest
pipeline = Pipeline(stages=[assembler, rndfr_sp])

# Treinar o modelo e prever as classes do conjunto de teste
model = pipeline.fit(df_train_aws)
df_pred1_aws = model.transform(df_test_aws)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Visualização dos Resultados
df_pred1_aws.select('features', 'rawPrediction', 'prediction', 'hotel_cluster').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----------+-------------+
|            features|       rawPrediction|prediction|hotel_cluster|
+--------------------+--------------------+----------+-------------+
|[4.0,2.0,3.0,198....|[0.05352712119434...|      64.0|           30|
|[4.0,14.0,1.0,214...|[0.05148676055818...|      64.0|           82|
|[8.0,2.0,3.0,66.0...|[0.20656119992842...|      91.0|           48|
|[8.0,2.0,3.0,66.0...|[0.20656119992842...|      91.0|            7|
|[8.0,2.0,3.0,66.0...|[0.20656119992842...|      91.0|           48|
|[8.0,2.0,3.0,66.0...|[0.20656119992842...|      91.0|           42|
|[8.0,2.0,3.0,66.0...|[0.20656119992842...|      91.0|            7|
|[8.0,2.0,3.0,66.0...|[0.20656119992842...|      91.0|            7|
|[8.0,2.0,3.0,66.0...|[0.20656119992842...|      91.0|            7|
|[11.0,2.0,3.0,66....|[0.20402639985294...|      91.0|           47|
|[11.0,37.0,1.0,69...|[0.18551255987846...|      91.0|           16|
|[16.0,2.0,3.0,66....|[0.206561199

In [15]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Define o Evaluator
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='hotel_cluster', metricName='accuracy')

# Calcula a accuracy, precisão, recall e f1-score
accuracy = evaluator.evaluate(df_pred1_aws)
precision = evaluator.evaluate(df_pred1_aws, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(df_pred1_aws, {evaluator.metricName: "weightedRecall"})
f1_score = evaluator.evaluate(df_pred1_aws, {evaluator.metricName: "f1"})

print("Accuracy = ", accuracy)
print("Precision = ", precision)
print("Recall = ", recall)
print("F1-Score = ", f1_score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy =  0.10161375198269955
Precision =  0.04777440959152639
Recall =  0.10161375198269952
F1-Score =  0.04332095620310921

----

## Decision Forest [[2]](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.DecisionTreeClassifier.html) [[3]](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html)

<div class="alert alert-block alert-warning">
Amostra do Dataset | <b>Computador</b></div>

- Com o `small_df_expedia` (no **Computador Local**)

In [16]:
from pyspark.ml.classification import DecisionTreeClassifier

# Criar um VectorAssembler para concatenar as colunas para o modelo
assembler = VectorAssembler(inputCols=features_list, outputCol='features')

# Criar um classificador Decision Tree
dt = DecisionTreeClassifier(featuresCol='features', labelCol='hotel_cluster')

# Pipeline com o VectorAssembler e o classificador Decision Tree
pipeline = Pipeline(stages=[assembler, dt])

# Treinar o modelo e prever as classes do conjunto de teste
model = pipeline.fit(df_train)
df_pred2_cl = model.transform(df_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# Visualização dos Resultados
df_pred2_cl.select('features', 'rawPrediction', 'prediction', 'hotel_cluster').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----------+-------------+
|            features|       rawPrediction|prediction|hotel_cluster|
+--------------------+--------------------+----------+-------------+
|[4.0,14.0,1.0,214...|[157.0,0.0,16459....|      82.0|           30|
|[4.0,30.0,4.0,195...|[157.0,0.0,16459....|      82.0|           25|
|[16.0,2.0,3.0,66....|[3713.0,0.0,7586....|      91.0|            7|
|[16.0,2.0,3.0,66....|[3713.0,0.0,7586....|      91.0|           76|
|[19.0,11.0,3.0,35...|[157.0,0.0,16459....|      82.0|           75|
|[21.0,13.0,1.0,46...|[6253.0,0.0,107.0...|       0.0|            3|
|[21.0,15.0,1.0,55...|[6253.0,0.0,107.0...|       0.0|           58|
|[21.0,15.0,1.0,55...|[6253.0,0.0,107.0...|       0.0|           62|
|[21.0,15.0,1.0,55...|[6253.0,0.0,107.0...|       0.0|           81|
|[21.0,23.0,1.0,1....|[6253.0,0.0,107.0...|       0.0|           81|
|[21.0,28.0,1.0,68...|[6253.0,0.0,107.0...|       0.0|            3|
|[21.0,37.0,1.0,69...|[6253.0,0.0,

In [18]:
# Define o Evaluator
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='hotel_cluster', metricName='accuracy')

# Calcula a Accuracy, Precision, Recall e F1-Score
accuracy = evaluator.evaluate(df_pred2_cl)
precision = evaluator.evaluate(df_pred2_cl, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(df_pred2_cl, {evaluator.metricName: "weightedRecall"})
f1_score = evaluator.evaluate(df_pred2_cl, {evaluator.metricName: "f1"})

print("Accuracy = ", accuracy)
print("Precision = ", precision)
print("Recall = ", recall)
print("F1-Score = ", f1_score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy =  0.09665977153661882
Precision =  0.03599310620320441
Recall =  0.09665977153661882
F1-Score =  0.03712287889029028

---

<div class="alert alert-block alert-danger">
Dataset Completo | <b>AWS</b></div>

- Com o `df_expedia` (no **AWS**)

In [19]:
from pyspark.ml.classification import DecisionTreeClassifier

# Criar um VectorAssembler para concatenar as colunas para o modelo
assembler = VectorAssembler(inputCols=features_list, outputCol='features')

# Criar um classificador Decision Tree
dt = DecisionTreeClassifier(featuresCol='features', labelCol='hotel_cluster')

# Pipeline com o VectorAssembler e o classificador Decision Tree
pipeline = Pipeline(stages=[assembler, dt])

# Treinar o modelo e prever as classes do conjunto de teste
model = pipeline.fit(df_train_aws)
df_pred2_aws = model.transform(df_test_aws)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Visualização dos Resultados
df_pred2_aws.select('features', 'rawPrediction', 'prediction', 'hotel_cluster').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----------+-------------+
|            features|       rawPrediction|prediction|hotel_cluster|
+--------------------+--------------------+----------+-------------+
|[4.0,2.0,3.0,198....|[536.0,0.0,55321....|      82.0|           30|
|[4.0,14.0,1.0,214...|[536.0,0.0,55321....|      82.0|           82|
|[8.0,2.0,3.0,66.0...|[19059.0,0.0,3029...|      91.0|           48|
|[8.0,2.0,3.0,66.0...|[19059.0,0.0,3029...|      91.0|            7|
|[8.0,2.0,3.0,66.0...|[19059.0,0.0,3029...|      91.0|           48|
|[8.0,2.0,3.0,66.0...|[19059.0,0.0,3029...|      91.0|           42|
|[8.0,2.0,3.0,66.0...|[19059.0,0.0,3029...|      91.0|            7|
|[8.0,2.0,3.0,66.0...|[19059.0,0.0,3029...|      91.0|            7|
|[8.0,2.0,3.0,66.0...|[19059.0,0.0,3029...|      91.0|            7|
|[11.0,2.0,3.0,66....|[19059.0,0.0,3029...|      91.0|           47|
|[11.0,37.0,1.0,69...|[19059.0,0.0,3029...|      91.0|           16|
|[16.0,2.0,3.0,66....|[19059.0,0.0

In [21]:
# Define o Evaluator
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='hotel_cluster', metricName='accuracy')

# Calcula a Accuracy, Precision, Recall e F1-Score
accuracy = evaluator.evaluate(df_pred2_aws)
precision = evaluator.evaluate(df_pred2_aws, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(df_pred2_aws, {evaluator.metricName: "weightedRecall"})
f1_score = evaluator.evaluate(df_pred2_aws, {evaluator.metricName: "f1"})

print("Accuracy = ", accuracy)
print("Precision = ", precision)
print("Recall = ", recall)
print("F1-Score = ", f1_score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy =  0.09730224773550628
Precision =  0.03761101110802035
Recall =  0.09730224773550628
F1-Score =  0.03809969947897531

---

### Conclusão dos Modelos

> Tanto com o *dataset* completo como com parte deste ($30\%$), os resultados dos modelos foram semelhantes, não tendo obtido um desempenho satisfatório para a tarefa de classificação do target `hotel_cluster`.

---

# 🛠️ Tunning do Modelo (Apenas no AWS)

> Como aconselhado pelo docente, efetuamos ***tunning*** ao modelo com melhor performance (de entre os dois modelos testados) de forma a tentar melhorá-lo com diferentes parametrizações.

Assim, fomos procurar formas de o fazer e codificar. [[4]](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.ParamGridBuilder.html)

In [22]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Criar um VectorAssembler para concatenar as colunas para o modelo
assembler = VectorAssembler(inputCols=features_list, outputCol='features')

# Criar um classificador Decision Tree
dt = DecisionTreeClassifier(featuresCol='features', labelCol='hotel_cluster')

# Pipeline com o VectorAssembler e o classificador Decision Tree
pipeline = Pipeline(stages=[assembler, dt])

# Definir a grade de parâmetros para testar
paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [2, 5, 10]) \
    .addGrid(dt.minInstancesPerNode, [1, 5, 10]) \
    .build()

# Usar CrossValidator para ajustar o modelo com diferentes combinações de parâmetros
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(labelCol='hotel_cluster'),
                          numFolds=5)

# Ajustar o modelo com os dados de treino
cvModel = crossval.fit(df_train_aws)

# Prever as classes do conjunto de teste usando o modelo ajustado
df_pred2_tunning = cvModel.transform(df_test_aws)

# Avaliar a precisão do modelo usando a métrica 'accuracy'
evaluator = MulticlassClassificationEvaluator(labelCol='hotel_cluster', metricName='accuracy')
accuracy = evaluator.evaluate(df_pred2_tunning)
precision = evaluator.evaluate(df_pred2_tunning, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(df_pred2_tunning, {evaluator.metricName: "weightedRecall"})
f1_score = evaluator.evaluate(df_pred2_tunning, {evaluator.metricName: "f1"})

print("Accuracy = ", accuracy)
print("Precision = ", precision)
print("Recall = ", recall)
print("F1-Score = ", f1_score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy =  0.13418873164868805
Precision =  0.17730201087290115
Recall =  0.13418873164868803
F1-Score =  0.08797882805318644

---

# 🔗 Referencias Bibliográficas

**[1]** Apache Spark. (n.d.-g). RandomForestClassifier — PySpark 3.3.2 documentation. Spark.apache.org. https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.RandomForestClassifier.html?highlight=randomforestclassifier

**[2]** Apache Spark. (n.d.-a). DecisionTreeClassifier — PySpark 3.3.2 documentation. Spark.apache.org. https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.DecisionTreeClassifier.html

**[3]** Apache Spark. (n.d.-b). MulticlassClassificationEvaluator — PySpark 3.3.2 documentation. Spark.apache.org. https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html

**[4]** Apache Spark. (n.d.-a). ParamGridBuilder — PySpark 3.3.2 documentation. Spark.apache.org. https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.ParamGridBuilder.html


